In [1]:
import cassandra
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json
import pandas as pd
import fiddler as fdl

In [2]:
cassandra.__version__

'3.28.0'

In [4]:
# This secure connect bundle is autogenerated when you donwload your SCB, 
# if yours is different update the file name below
cloud_config= {
  'secure_connect_bundle': 'datastax_auth/secure-connect-fiddlerai.zip'
}

# This token json file is autogenerated when you donwload your token, 
# if yours is different update the file name below
with open("datastax_auth/danny@fiddler.ai-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

In [5]:
session.set_keyspace('fiddlerai')

In [6]:
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row_factory = pandas_factory
session.default_fetch_size = None

In [21]:
rows = session.execute('SELECT * FROM fiddler_doc_snippets_openai')
    
df_docs = rows._current_rows
df_docs

,row_id,attributes_blob,body_blob,metadata_s,vector
0,92623507537844779456935d731166a7,None,"---\ntitle: ""Fairness""\nslug: ""fairness""\nhidd...",[embedding],"[-0.022576844319701195, -0.009307874366641045,..."
1,f37ba2e2aa5f47c4afe7ede8e9896e05,None,"---\ntitle: ""Useful Queries for Root Cause Ana...",[embedding],"[0.008825183846056461, 0.027762411162257195, -..."
2,4c4d2927e9b9400d80596a0e71d7d5f0,None,"slug: ""clientpublish_events_batch"" _S3"",\n ...",[embedding],"[-0.02316180244088173, -0.011533494107425213, ..."
3,70a6ea7c2cfa4fdb96ebb3beda022c24,None,"---\ntitle: ""Performance Tracking""\nslug: ""per...",[embedding],"[-0.013761984184384346, -0.005487993825227022,..."
4,19eaf41707e3460284184577ad55819d,None,"slug: ""evaluation-ui"" Calibration Plot_**\n -...",[embedding],"[-0.019045107066631317, -0.005970435217022896,..."
...,...,...,...,...,...
625,fd34d0931c374ebe9420b3869e1fc567,None,"slug: ""publishing-events-with-complex-data-for...",[embedding],"[-0.009095759131014347, 0.018063507974147797, ..."
626,6caa8cf2fe60426a9e6d32fa88bbdd77,None,"---\ntitle: ""fdl.AlertType""\nslug: ""fdlalertty...",[embedding],"[-0.0012485459446907043, 0.01317746564745903, ..."
627,b4c745fbd1d1461bb937821e9fe663dc,None,"slug: ""customer-churn-prediction"" Churn-image5...",[embedding],"[0.0011718893656507134, 0.000443618802819401, ..."
628,3c34de11ae5a40c89e0889f105030519,None,"---\ntitle: ""client.build_notifications_config...",[embedding],"[-0.020654888823628426, 0.003742130473256111, ..."


In [30]:
rows = session.execute('SELECT row_id, session_id, source_docs, response, question, comment, feedback, ts FROM fiddler_chatbot_history')
    
df = rows._current_rows
df

,row_id,session_id,source_docs,response,question,comment,feedback,ts
0,c5c14f05-df8c-402c-9fd3-3af8f6999231,fefc3bb2-724a-43ae-953f-651351a5a5f2,"Document: ---\ntitle: ""Baselines""\nslug: ""f...",There is no specific mention of the maximum nu...,How many of them can I have?,None,1,2023-10-12 13:44:06.765
1,1e21aade-7d3a-4ba5-9c30-3bcd59e55fe5,fefc3bb2-724a-43ae-953f-651351a5a5f2,"Document: ---\ntitle: ""Baselines""\nslug: ""f...","Yes, you can have multiple baselines in Fiddle...",Can I have multiple baselines?,Great,1,2023-10-12 13:43:17.893


In [16]:
#session.execute("DELETE FROM fiddler_chatbot_conversation WHERE row_id='-1'")

In [35]:
#session.execute("DROP TABLE fiddler_chatbot_history")

In [23]:
#session.execute("TRUNCATE TABLE fiddler_doc_snippets_openai")

In [27]:
session.execute("TRUNCATE TABLE fiddler_chatbot_history")